<a href="https://colab.research.google.com/github/jorge774/Procesamiento_datos/blob/main/procesar_datos_v3.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''procesar_datos.py
   Este programa hace lo siguiente:
   1.Reavisa la carpeta donde estan los archivos txt que genera janus luego de hacer adquisicion.
   2.Revisa y toma los archivos txt que no ha leido anteriormente.
   3.Toma el encabezado del archivo RunXX_list.txt con menor XX de todo el lote de archivos tomados en 2.
   4.Toma como referencia temporal el 'Run start time' del encabezado tomado en 3.
   5.Genera una estampa temporal para cada evento de todos los archivos RunXX_list.txt tomando como referencia a 4.
   6.Genera un archivo 'MuTe_20240304_11h07m01s.txt'
   7.Elimina aquellos eventos donde el conteo en todos los canales fue 0.
   8.Elimina los archivos RunXX_list.txt

   Los parametros de este programa son leidos en el archivo 'config.txt', los cuales son:
   1.Ruta absoluta donde quedan los archivos txt que genera janus luego de hacer adquisicion.
   2.Ruta absoluta donde quedaran los archivos 'MuTe_20240304_11h07m01s.txt'.
   3.Numero de canales usados en la adquision.
   4.Numero de lineas usadas en los encabezados de los archivos 'RunXX_list.txt'.

   En el archivo 'vars.txt' el programa escribe el nombre de los archivos 'RunXX_list.txt' que va procesando.
'''

In [ ]:
print('Procesando...')
import os
import sys
import re
import time
from datetime import datetime
from datetime import timedelta
from pytz import timezone
import pandas as pd
import tarfile
ini_exe=datetime.now(timezone('America/Bogota'))
config_file=open(file='config.txt',mode='r');output_input_folders=config_file.read();config_file.close()
num_header_lines=int(output_input_folders.split('\n')[3].split('#')[0])#8
#n_channels=int(output_input_folders.split('\n')[2].split('#')[0])#64 #numero de canales
folder_path_caen=output_input_folders.split('\n')[0].split('#')[0]
output_folder_path=output_input_folders.split('\n')[1].split('#')[0]
pos_mute=output_input_folders.split('\n')[4].split('#')[0]
padding=sys.argv[1].split()[0]#hacer padding en el txt, string s o n
csv=sys.argv[1].split()[1]#generar csv string s o n
batch=int(sys.argv[1].split()[2])#tamaño del lote de archivos de corrida

In [ ]:
#gestor de errores en los archivos run list

#eliminar archivos.dat

#ordenar los files_to_proces de acuerdo al run start time
def key_function(file,folder_path_caen):
    if re.search('list.txt',file):
       file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
       content=file_to_proces.readlines()[:10];file_to_proces.close()
       string_date_time=''
       for i in content[6].split()[4:-1]:i+=' ';string_date_time+=i
       #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
       ref_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
       ref_date_time=pd.Timestamp(str(ref_date_time))
       del content
    if re.search('Info.txt',file):
       file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
       content=file_to_proces.readlines()[:10];file_to_proces.close()
       ref_date_time=content[3].split()[2]+' '+content[3].split()[3]
       ref_date_time=pd.Timestamp(datetime.strptime(ref_date_time,'%d/%m/%Y %H:%M'))
       del content
    #ignorar folders y archivos que no sean de info o corrida
    if not re.search('list.txt',file) and not re.search('Info.txt',file):ref_date_time=pd.Timestamp('2030-01-01 00:00:00')
    return ref_date_time

#verificar que un archivo de corrida tenga informacion de todos los canales en todas las ventanas de adquision
def check_arvhivo_corrida(file,folder_path_caen,num_header_lines,n_channels):
    print('Verificando archivo de corrida...')
    check=True#archivo de corrida sin errores
    file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
    data_and_header=file_to_proces.readlines();file_to_proces.close()
    ini_index=num_header_lines+1
    end_index=num_header_lines+n_channels+1
    while ini_index<(len(data_and_header)-1):
          #recorrer todos los data blocks de un archivo Runxx_list.txt
          data_block=data_and_header[ini_index:end_index]
          chs_in_acq=[int(i.split()[1]) for i in data_block]
          if len(chs_in_acq)!=n_channels:
             check=False#archivo de corrida con errores
             #os.remove(file_path)#eliminar archivo runxxlits.txt corrupto
             print('El archivo '+file+' esta corrupto.')
             break
          ini_index=end_index
          end_index=ini_index+n_channels
    del data_and_header
    return check

#infiere el numero de chs en adquision. Supone que se adquirió almenos un evento
def inferir_n_channels(file,folder_path_caen,num_header_lines):
    #infiere el numero de chs en adquision. Supone que se adquirió almenos un evento
    file_path=folder_path_caen+'/'+file;file_to_proces=open(file=file_path,mode='r')
    data_and_header=file_to_proces.readlines();file_to_proces.close()
    n_channels=1;ch_index=num_header_lines+2
    while len(data_and_header[ch_index].split())<=3:
          n_channels+=1;ch_index+=1
          try:
            data_and_header[ch_index]
          except IndexError:
                break
    print('El archivo '+file+' tiene informacion de '+str(n_channels)+' canales.')
    return n_channels

#cambiar el nombre de los archivos Runx_List.txt e Runx_Info.txt
def rename_file_folder_path_caen(folder_path_caen,file,current_time_stamp):
    #1.verificar si es un archivo info o un archivo run_list
    #2.generar string con la fecha y hora
    #3.cambiar el nobre
    file_path=folder_path_caen+'/'+file
    if re.search('list.txt',file):
       new_name=folder_path_caen+'/'+'MUTE'+'_'+pos_mute+'_'+current_time_stamp+'.raw'
       os.rename(file_path,new_name)
    if re.search('Info.txt',file):
       new_name=folder_path_caen+'/'+'MUTE'+'_'+pos_mute+'_'+current_time_stamp+'.raw.mtd'
       os.rename(file_path,new_name)

#se asume que para cada archivo de corrida hay un archivo de info
def armar_lote_archivos_de_corrida(batch):
    temp=[];batch_info=batch;batch_run=batch
    for i in files_to_proces:
        if re.search('list.txt',i) and batch_run>0:
           batch_run-=1;temp.append(i)
        if re.search('Info.txt',i) and batch_info>0:
           batch_info-=1;temp.append(i)
    return temp

In [ ]:
folder_path_vars='vars.txt'
##ini main loop
vars_file=open(file=folder_path_vars,mode='r')
processed_file_list=vars_file.read()
vars_file.close()
current_file_list=os.listdir(folder_path_caen)

if processed_file_list:
   #processed_file_list is not empty
   files_to_proces=list(set(current_file_list)-set(processed_file_list.split('\n')))#archivos que esten en folder_path_caen pero que no esten en el vars file
else:
   #processed_file_list is empty
   files_to_proces=current_file_list

files_to_proces.sort(key=lambda file:key_function(file,folder_path_caen))#ordenar los files_to_proces de acuerdo al run start time

#aqui se agarra x cantidad de archivos run_list y x cantidad de archivos Info para procesarlos
files_to_proces=armar_lote_archivos_de_corrida(batch)

if files_to_proces:
    #files_to_proces is not empty
    count_files_run_list_processed=0
    #formato v1:'MuTe_20240304_11h07m01s'
    #formato v2:MUTE_MACH1_DATE-TIME.dat
    output_file_path=''
    gmt_5_datetime=datetime.now(timezone('America/Bogota'))
    cur_h='0'+str(gmt_5_datetime.hour) if gmt_5_datetime.hour<10 else str(gmt_5_datetime.hour)
    cur_day='0'+str(gmt_5_datetime.day) if gmt_5_datetime.day<10 else str(gmt_5_datetime.day)
    cur_mon='0'+str(gmt_5_datetime.month) if gmt_5_datetime.month<10 else str(gmt_5_datetime.month)
    cur_min='0'+str(gmt_5_datetime.minute) if gmt_5_datetime.minute<10 else str(gmt_5_datetime.minute)
    cur_sec='0'+str(gmt_5_datetime.second) if gmt_5_datetime.second<10 else str(gmt_5_datetime.second)
    current_time_stamp=str(gmt_5_datetime.year)+cur_mon+cur_day+'_'+cur_h+'h'+cur_min+'m'+cur_sec+'s'
    name='MUTE'+'_'+pos_mute+'_'+current_time_stamp
    #name='MUTE_MACH1_'+str(gmt_5_datetime.year)+cur_mon+cur_day+'_'+cur_h+'h'+cur_min+'m'+cur_sec+'s'
    files_to_compress=[]
    print('Escribiendo archivo de salida...')
    for file in files_to_proces:
        print('Leyendo archivo '+file+'...')
        if re.search('list.txt',file):n_channels=inferir_n_channels(file,folder_path_caen,num_header_lines)
        if re.search('list.txt',file) and check_arvhivo_corrida(file,folder_path_caen,num_header_lines,n_channels):
           #procesar los datos adquiridos
           #se procesa un archivo Runxx_list.txt de files_to_proces
           #ejecutar con cada archivo Runxx_list.txt
           file_path=folder_path_caen+'/'+file
           file_to_proces=open(file=file_path,mode='r')
           data_and_header=file_to_proces.readlines()
           file_to_proces.close()
           #current_time_stamp debe ser la que trae el archivo de corrida
           file_time_stamp=key_function(file,folder_path_caen)
           file_time_stamp_str=str(file_time_stamp.year)+str(file_time_stamp.month)+str(file_time_stamp.day)+'_'+str(file_time_stamp.hour)+'h'+str(file_time_stamp.minute)+'m'+str(file_time_stamp.second)+'s'
           #cambiar nombre de archivos de corrida
           rename_file_folder_path_caen(folder_path_caen,file,file_time_stamp_str)
           output_file_path=output_folder_path+'/'+name+'.dat'
           output_file=open(file=output_file_path,mode='a')

           header=data_and_header[:num_header_lines]
           if count_files_run_list_processed==0:
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              ref_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
              ref_date_time=pd.Timestamp(str(ref_date_time))
              delta_time=ref_date_time-ref_date_time
              temp=data_and_header[num_header_lines+1:num_header_lines+n_channels+1]
              chs_in_acq=[int(i.split()[1]) for i in temp]
              aux=['ch0'+str(i) if i<10 else 'ch'+str(i) for i in chs_in_acq]
              aux2=[aux[i]+' ' if i!=(len(aux)-1) else aux[i] for i in range(len(aux))]
              header2='Tstamp GMT-5(aa-mm-dd hh:mm:ss.ns) '
              for i in aux2:header2+=i
              output_file.write(header2+'\n')
           else:
              string_date_time=''
              for i in header[6].split()[4:-1]:i+=' ';string_date_time+=i
              #datetime.strptime('Fri Mar 1 16:32:05 2024 ','%a %b %d %H:%M:%S %Y ')
              last_date_time=datetime.strptime(string_date_time,'%a %b %d %H:%M:%S %Y ')-timedelta(hours=5)
              last_date_time=pd.Timestamp(str(last_date_time))
              delta_time=last_date_time-ref_date_time

           ini_index=num_header_lines+1
           end_index=num_header_lines+n_channels+1
           while ini_index<(len(data_and_header)-1):
                #recorrer todos los data blocks de un archivo Runxx_list.txt
                data_block=data_and_header[ini_index:end_index]
                #ejeutar para cada data block
                ch_counts=[]
                for i in range(len(data_block)):ch_counts.append(data_block[i].split()[2])
                if ch_counts.count('0')!=n_channels:#verificar que almenos un ch se haya activado
                    Tstamp_us=data_block[0].split()[3]
                    nanosegundos=(delta_time.total_seconds()*1e+9)+(float(Tstamp_us)*1000)
                    Tstamp_gmt_5=ref_date_time+pd.Timedelta(nanosegundos,unit='ns')
                    output_file.write(str(Tstamp_gmt_5))
                    for _ in range(36-len(str(Tstamp_gmt_5))):output_file.write(' ')
                    for i in range(len(data_block)):
                        count=data_block[i].split()[2]
                        if i!=(len(data_block)-1):
                           output_file.write(count)
                           for _ in range(4+1-len(list(count))):output_file.write(' ')
                        else:
                           output_file.write(count+'\n')
                ini_index=end_index
                end_index=ini_index+n_channels
           output_file.close()
           count_files_run_list_processed+=1
           files_to_compress.append(folder_path_caen+'/'+'MUTE'+'_'+pos_mute+'_'+file_time_stamp_str+'.raw')#archivo de corrida marcado para comprimir
           #os.remove(file_path)#eliminar archivo runxxlits.txt
           del data_and_header#liberar memoria
        if re.search('Info.txt',file):
           #procesar metadata de los datos adquiridos
           #current_time_stamp debe ser la que trae el archivo de Info
           file_time_stamp=key_function(file,folder_path_caen)
           file_time_stamp_str=str(file_time_stamp.year)+str(file_time_stamp.month)+str(file_time_stamp.day)+'_'+str(file_time_stamp.hour)+'h'+str(file_time_stamp.minute)+'m'+str(file_time_stamp.second)+'s'
           #cambiarn nombre de archivo de Info
           rename_file_folder_path_caen(folder_path_caen,file,file_time_stamp_str)
           files_to_compress.append(folder_path_caen+'/'+'MUTE'+'_'+pos_mute+'_'+file_time_stamp_str+'.raw.mtd')#archivo de Info marcado para comprimir
    if os.path.exists(output_file_path):
        output_file=open(file=output_file_path,mode='r');content_output_file=output_file.readlines();output_file.close()
        if not content_output_file[1:]:
           os.remove(output_file_path)#eliminar txt si esta vacio
        else:
            if csv=='s' or padding=='s':
                #aqui si arma un csv con los datos
                all_64_channels=64
                aux=['ch0'+str(i) if i<10 else 'ch'+str(i) for i in range(all_64_channels)]
                chs_in_acq=content_output_file[0].split()[3:]
                chs_in_acq_num=[int(i[2:]) for i in chs_in_acq]
                event_values=content_output_file[1:]
                data=[];to_txt=[]
                for k in event_values:
                    chs_in_acq_values=[int(l) for l in k.split()[2:]]
                    all_chs_values=[];j=0
                    for i in range(all_64_channels):
                        if not i in chs_in_acq_num:
                          #ejecutar si i no esta en chs_in_acq_num
                          all_chs_values.append(0)
                        else:
                          #ejecutar si i esta en chs_in_acq_num
                          all_chs_values.append(chs_in_acq_values[j])
                          j+=1
                    all_chs_values_str=''
                    for t in range(len(all_chs_values)):
                        if  t!=(all_64_channels-1):
                            all_chs_values_str=all_chs_values_str+str(all_chs_values[t])+'    '
                        else:
                            all_chs_values_str=all_chs_values_str+str(all_chs_values[t])
                    row=k[:36]+all_chs_values_str+'\n'
                    to_txt.append(row)
                    if csv=='s':
                      #filas del dataframe
                      row_data_frame=[pd.Timestamp(row.split()[0]+' '+row.split()[1])]+[int(g) for g in row.split()[2:]]
                      data.append(row_data_frame)
                      #filas del dataframe
                if csv=='s':
                    print('Armando csv...')
                    name_columns=['time']+aux
                    df=pd.DataFrame(data,columns=name_columns)#armar dataframe
                    csv_output_name=output_folder_path+'/'+name+'.csv'
                    df.to_csv(csv_output_name,index=False)
                    files_to_compress.append(csv_output_name)#archivo csv de salida marcado para comprimir
                    del data
                if padding=='s':
                    #se sobre escribe el txt de salida con paddings
                    print('Rellenando con 0 los chs que no se adquirieron.')
                    aux2=[aux[i]+' ' if i!=(len(aux)-1) else aux[i] for i in range(len(aux))]
                    header2='Tstamp GMT-5(aa-mm-dd hh:mm:ss.ns) '
                    for i in aux2:header2+=i
                    header2=header2+'\n'
                    to_txt.insert(0,header2)
                    output_file=open(file=output_file_path,mode='w');output_file.writelines(to_txt);output_file.close()
                    del to_txt
            files_to_compress.append(output_file_path)#archivo de salida marcado para comprimir
        del content_output_file
    vars_file=open(file=folder_path_vars,mode='a')
    if processed_file_list:
       #processed_file_list is not empty
       vars_file.write('\n')
       for i in range(0,len(files_to_proces)):
           if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
           else:
              vars_file.write(files_to_proces[i])
    else:
      #processed_file_list is empty
      for i in range(0,len(files_to_proces)):
          if i!=(len(files_to_proces)-1):
              vars_file.write(files_to_proces[i]+'\n')
          else:
              vars_file.write(files_to_proces[i])
    vars_file.close()
    #Generar metadata del archivo .dat que creó este codigo.
    #En este punto del codigo se genera un archivo de metada por lote de archivos de corrida procesado
    meta_data_file_path=output_folder_path+'/'+name+'.dat.mtd'

    n_raws_files=0;n_raws_mtd_files=0
    for j in range(len(files_to_compress)):
        if re.findall('.raw$',files_to_compress[j]):n_raws_files+=1
        if re.findall('.raw.mtd$',files_to_compress[j]):n_raws_mtd_files+=1
    content_tar_file='Este archivo tar contiene '+str(len(files_to_compress)+1)+' archivos:\n'\
                    +'-El presente archivo de metadata de el/los archivo/os crudo/os procesado/os(dat.mtd)\n'\
                    +'-1 archivo de datos crudos procesados(.dat)\n'\
                    +'-'+str(n_raws_files)+' archivo/os de dato/os crudo/os(.raw)\n'\
                    +'-'+str(n_raws_mtd_files)+' archivo/os de metada de archivo/os crudo/os(.raw.mtd)\n'
    meta_data=[content_tar_file,'Codigo usado para procesar datos crudos: https://gitmilab.redclara.net/muografia/MuTeInstrument/mute-2024-analisis-codigos.git\n'\
              ,'Desarrollado por: Jorge L. Perea\n','Correo: jorge.perea@correo.uis.edu.co\n']
    if csv=='s':meta_data.insert(1,'-1 archivo csv que contiene la misma informacion del .dat\n')
    meta_data_file=open(file=meta_data_file_path,mode='w');meta_data_file.writelines(meta_data);meta_data_file.close()

    files_to_compress.append(meta_data_file_path)#archivo de metada del arvhivo de salida marcado para compromir
    #Comprimir archivos
    #files_to_compress contiene:
    #-archivo .raw
    #-archivo .raw.mtd
    #-archivo .dat
    #-archivo .dat.mtd
    print('Comprimiendo archivos...')
    tar = tarfile.open(output_folder_path+'/'+name+'.tar.gz','w:gz')
    for file in files_to_compress:
        tar.add(file,arcname=os.path.basename(file))
        os.remove(file)
    tar.close()
print('Procesamiento terminado. Duracion del procesamiento:',(datetime.now(timezone('America/Bogota'))-ini_exe).total_seconds(),'segundos')
##end main loop

In [ ]:
#generar pixel map